# ReAct

在function_calling应用场景中，如果用户在单次请求中包含多个意图时，e.g. `你喜欢旅游吗？上海有什么景点？`。这种复杂场景中，单纯依靠原始的提示词，LLM无法完美实现function_calling的调用(通过分析问题决定when&how调用tools)。

AI agent能自主感知、做出决策并执行行动的复合体。把AI Agent想象为一个能够独立思考和行动的人。

论文[《ReAct：Synergizing Reasoning and Acting in Language Models》](https://arxiv.org/pdf/2210.03629)，是一个prompt框架。示例：

In [ ]:
prompt = """

You run in a loop of Thought, Action, Observation, Answer.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you.
Observation will be the result of running those actions.
Answer will be the result of analysing the Observation

Your available actions are:

xiaohongshu:
e.g. xiaohongshu: Beijing travel tips
Runs a search through the Xiaohongshu API and returns travel tips and recommendations for Beijing.

ctrip:
e.g. ctrip: flights to Beijing
Runs a search through the Ctrip API to find available flights to Beijing.

Always use the Xiaohongshu and Ctrip APIs if you have the opportunity to do so.

Example session:

Question: I'm planning a trip to Beijing, what should I do first?

Thought: I should find out about the attractions and tips for visiting Beijing on Xiaohongshu.

Action: xiaohongshu: Beijing travel tips

Observation: The search returns a list of popular travel tips and must-visit attractions in Beijing.

Answer: Start by researching Beijing's must-visit attractions and travel tips on Xiaohongshu. Then, look for available flights on Ctrip and consider accommodation options.

....

"""

1. prompt定义了 `thought - action - observation - answer`的loop
2. 定义tools及用法
3. loop的演示 #one-shot

instead of 单一的任务描述，这样一个更复杂的prompt，定义了agent的具体职责、完成这个任务所需采取的操作，以及这些操作所具备的能力。
TODO: reference [ReAct github](https://react-lm.github.io/)的prompt

ReAct = Reason + Act
* Reason, 基于思维链(CoT, chain of thought)，将输入分解为多个逻辑思维步骤：
    * 分解问题，面对复杂任务时，CoT方法不是通过单个步骤解决它，而是将任务分解为多个子任务，每个子任务解决一个小问题。
    * 顺序，CoT的每个步骤都建立在上一步的结果之上，这样模型能构造出一条完整的逻辑推理链。

![CoT](./imgs/CoT.png)

上图可以看出，将推理过程告知LLM，能够获得正确答案。
但是，LLM仍然会产生幻觉。在推理的中间阶段会产生不正确的答案，获上下游传播错误数据。
so, DeepMind团队研发了ReAct，弥补CoT的缺点。ReAct采用 thought-action-observation 的loop，agent通过观察上一个步骤的结果来进行推理和行动。
![ReAct](./imgs/ReAct.png)

其中,`action_input`是LLM与外部环境的实时交互，e.g. 搜索引擎、调用api.
另一个ReAct框架的prompt示例：

In [ ]:
prompt = """
You run in a loop of Thought, Action, Observation, Answer.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you.
Observation will be the result of running those actions.
Answer will be the result of analysing the Observation

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?

Thought: I should look up France on Wikipedia

Action: wikipedia: France

You should then call the appropriate action and determine the answer from 
the result

You then output:

Answer: The capital of France is Paris
"""

## 代码演示
来自某机构课程

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
system_prompt = """
You run in a loop of Thought, Action, Observation, Answer.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you.
Observation will be the result of running those actions.
Answer will be the result of analysing the Observation

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

fetch_real_time_info:
e.g. fetch_real_time_info: Django
Returns a real info from searching SerperAPI

Always look things up on fetch_real_time_info if you have the opportunity to do so.

Example session:

Question: What is the capital of China?
Thought: I should look up on SerperAPI
Action: fetch_real_time_info: What is the capital of China?
PAUSE 

You will be called again with this:

Observation: China is a country. The capital is Beijing.
Thought: I think I have found the answer
Action: Beijing.
You should then call the appropriate action and determine the answer from the result

You then output:

Answer: The capital of China is Beijing

Example session

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth on fetch_real_time_info
Action: fetch_real_time_info : mass of earth
PAUSE

You will be called again with this: 

Observation: mass of earth is 1,1944×10e25

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

1. prompt定义了 `thought - action - observation - answer`的loop
2. 声明了2个tools及用法，calculate、fetch_real_time_info
3. loop的演示 #few-shot

In [18]:
import requests
import os

def fetch_real_time_info(query):
    # API参数
    params = {
        'api_key': os.getenv('SERP_API_KEY'), 
        'engine' : "google",
        'q': query,    # 查询参数，表示要搜索的问题。
        'num': 1       # 返回结果的数量设为1，API将返回一个相关的搜索结果。
    }

    # 发起GET请求到Serper API
    api_result = requests.get('https://serpapi.com/search', params)
    
    # 解析返回的JSON数据
    search_data = api_result.json()
    # print(f'search_data: {search_data}')
    # 提取并返回查询到的信息
    if 'organic_results' in search_data:
        return search_data["organic_results"][0]["snippet"]
    else:
        return "没有找到相关结果。"

In [19]:
# 测试代码
query = "世界上最长的河流是哪条河流？"
result = fetch_real_time_info(query)
print(result)

1. 尼罗河（Nile）. 6670 ; 2. 亚马逊河（Amazon）. 6400 ; 3. 长江（Chang Jiang）. 6397 ; 4. 密西西比河（Mississippi）. 6020 ; 5. 叶尼塞河（Yenisey）. 5539.


In [20]:
# 函数 `calculate` 接收一个字符串参数 operation，该字符串代表一个数学运算表达式，
# 并使用 Python 的内置函数 eval 来执行这个表达式，然后返回运算的结果。
# 函数的返回类型被指定为 float，意味着期望返回值为浮点数。
def calculate(operation: str) -> float:
    return eval(operation)

In [21]:
# 测试代码
result = calculate("100 / 5")
print(result)  # 输出结果应该是 20.0

20.0


In [22]:
# 存储函数的字典
available_actions = {
    "fetch_real_time_info": fetch_real_time_info,
    "calculate": calculate,
}

In [31]:
from openai import OpenAI

class ChatBot:
    def __init__(self, system=""):
        self.client = OpenAI()
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        completion = self.client.chat.completions.create(model="gpt-4o-mini", messages=self.messages)
        return completion.choices[0].message.content

代码解释：
#使用示例演示
```
bot = ChatBot(system="你是一个有帮助的助手")  # 初始化，可设置系统角色 调用--> __init__

response = bot("你好")  # 发送用户消息并获取回复 调用--> __call__
```

捕获action调用时机。
大模型识别到用户的意图中需要调用工具，那么其停留的阶段一定是在 Action：xxxx : xxxx 阶段，其中第一个 xxx，就是调用的函数名称，第二个 xxxx，就是调用第一个 xxxx 函数时，需要传递的参数。这里就可以通过正则表达式来进行捕捉。如下所示：

In [ ]:
import re
# (\w+) 是一个捕获组，匹配一个或多个字母数字字符（包括下划线）。这部分用于捕获命令中指定的动作名称
# (.*) 是另一个捕获组，它匹配冒号之后的任意字符，直到字符串结束。这部分用于捕获命令的参数。
# r means raw string, 意思是将\w直接传递给re.compile，而不会被python转义。
action_re = re.compile(r'^Action: (\w+): (.*)$')

In [28]:
# 测试代码
match = action_re.match("Action: fetch_real_time_info: mass of earth")
if match:
    print(match.group(1))  # 'fetch_real_time_info'
    print(match.group(2))  # 'mass of earth'

fetch_real_time_info
mass of earth


In [34]:
def AgentExecutor(question, max_turns=5):
    i = 0
    bot = ChatBot(system_prompt)
    # 通过 next_prompt 标识每一个子任务的阶段性输入
    next_prompt = question
    while i < max_turns:
        i += 1
        # 这里调用的就是 ChatBot 类的 __call__ 方法
        result = bot(next_prompt)
        print(f"{result}")
        # 在这里通过正则判断是否到了需要调用函数的Action阶段
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            # 提取调用的工具名和工具所需的入参
            action, action_input = actions[0].groups()
            if action not in available_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(f"running: {action} {action_input}")
            observation = available_actions[action](action_input)
            print(f"Observation: {observation}")
            next_prompt = "Observation: {}".format(observation)
        else:
            return bot.messages

In [35]:
AgentExecutor("世界上最长的河流是什么？")

Thought: 我应该在SerperAPI上查找世界上最长的河流的最新信息。  
Action: fetch_real_time_info: 世界上最长的河流是什么。  
PAUSE
running: fetch_real_time_info 世界上最长的河流是什么。  
Observation: 一、尼罗河. 尼罗河 ; 6650公里(4132英里)。其水资源共享的11个国家,即坦桑尼亚、乌干达、卢旺达、布隆迪、刚果民主共和国、肯尼亚、埃塞俄比亚、厄立特里亚、南苏丹、苏丹 ...
Thought: 我找到了答案，尼罗河被认为是世界上最长的河流，长度为6650公里。  
Action: 输出答案。  
Answer: 世界上最长的河流是尼罗河，长度为6650公里。


[{'role': 'system',
  'content': "You run in a loop of Thought, Action, Observation, Answer.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you.\nObservation will be the result of running those actions.\nAnswer will be the result of analysing the Observation\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nfetch_real_time_info:\ne.g. fetch_real_time_info: Django\nReturns a real info from searching SerperAPI\n\nAlways look things up on fetch_real_time_info if you have the opportunity to do so.\n\nExample session:\n\nQuestion: What is the capital of China?\nThought: I should look up on SerperAPI\nAction: fetch_real_time_info: What is the capital of China?\nPAUSE \n\nYou will be called again with this:\n\nObservation: China is a 

In [36]:
AgentExecutor("20 * 15 等于多少")

Thought: I need to perform a multiplication calculation to find the result of 20 times 15. 
Action: calculate: 20 * 15 
PAUSE
running: calculate 20 * 15 
Observation: 300
Thought: I have the result of the calculation, which is 300. 
Action: 300 
PAUSE


[{'role': 'system',
  'content': "You run in a loop of Thought, Action, Observation, Answer.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you.\nObservation will be the result of running those actions.\nAnswer will be the result of analysing the Observation\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nfetch_real_time_info:\ne.g. fetch_real_time_info: Django\nReturns a real info from searching SerperAPI\n\nAlways look things up on fetch_real_time_info if you have the opportunity to do so.\n\nExample session:\n\nQuestion: What is the capital of China?\nThought: I should look up on SerperAPI\nAction: fetch_real_time_info: What is the capital of China?\nPAUSE \n\nYou will be called again with this:\n\nObservation: China is a 

In [38]:
AgentExecutor("世界上最长的河流，与中国最长的河流，它们之间的差值是多少？")

Thought: 我需要查找世界上最长的河流以及中国最长的河流的信息，才能计算它们之间的差值。 
Action: fetch_real_time_info: 世界上最长的河流和中国最长的河流分别是哪些？ 
PAUSE
running: fetch_real_time_info 世界上最长的河流和中国最长的河流分别是哪些？ 
Observation: 1, 尼罗河－卡盖拉河, 6650 ; 2, 亚马逊河－乌卡亚利河－阿普里马克河, 6448 ; 3, 长江－通天河－沱沱河, 6380 ; 4, 密西西比河－密苏里河－红岩河（英语：Red Rock River (Montana)） ...
Thought: 我发现尼罗河是世界上最长的河流，长度为6650公里，而长江是中国最长的河流，长度为6380公里。接下来我需要计算这两条河流之间的差值。 
Action: calculate: 6650 - 6380 
PAUSE
running: calculate 6650 - 6380 
Observation: 270
Thought: 差值计算得出270公里，这是尼罗河与长江之间的长度差。 
Action: 270
Answer: 世界上最长的河流与中国最长的河流之间的差值是270公里。


[{'role': 'system',
  'content': "You run in a loop of Thought, Action, Observation, Answer.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you.\nObservation will be the result of running those actions.\nAnswer will be the result of analysing the Observation\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nfetch_real_time_info:\ne.g. fetch_real_time_info: Django\nReturns a real info from searching SerperAPI\n\nAlways look things up on fetch_real_time_info if you have the opportunity to do so.\n\nExample session:\n\nQuestion: What is the capital of China?\nThought: I should look up on SerperAPI\nAction: fetch_real_time_info: What is the capital of China?\nPAUSE \n\nYou will be called again with this:\n\nObservation: China is a 

## 流行框架

Langchain、LlamaIndex、CrewAI中集成了ReAct思想。

Langchain中的示例：

Answer the following questions as best you can. You have access 
to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}

There are three placeholders {tool}, {input}, and {agent_scratchpad} in this prompt. These will be replaced with the appropriate text before sending it to LLM.